In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
movie = pd.read_csv('./data/movie.csv')
movie.head() #영화 평점 관련 데이터

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
movie.info() #결측치 확인에 유용 
# 4916개의 entries 와 28개의 열을 확인해서 모자란 부분이 결측치임을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4916 entries, 0 to 4915
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   color                      4897 non-null   object 
 1   director_name              4814 non-null   object 
 2   num_critic_for_reviews     4867 non-null   float64
 3   duration                   4901 non-null   float64
 4   director_facebook_likes    4814 non-null   float64
 5   actor_3_facebook_likes     4893 non-null   float64
 6   actor_2_name               4903 non-null   object 
 7   actor_1_facebook_likes     4909 non-null   float64
 8   gross                      4054 non-null   float64
 9   genres                     4916 non-null   object 
 10  actor_1_name               4909 non-null   object 
 11  movie_title                4916 non-null   object 
 12  num_voted_users            4916 non-null   int64  
 13  cast_total_facebook_likes  4916 non-null   int64

#### 결측치, 이상치 제거

In [5]:
movie.isnull().sum()

color                         19
director_name                102
num_critic_for_reviews        49
duration                      15
director_facebook_likes      102
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        862
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                152
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               300
budget                       484
title_year                   106
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 326
movie_facebook_likes           0
dtype: int64

In [6]:
movie.shape

(4916, 28)

In [7]:
movie.dropna().shape 
#너무 많은 데이터가 제거됨, fillna 이후 dropna 진행

(3655, 28)

In [8]:
movie.color.value_counts() #범주형 데이터이므로 value_counts를 먼저 진행
#대부분 color 데이터이므로 결측치를 color로 대체

Color              4693
Black and White     204
Name: color, dtype: int64

In [10]:
col = {'color':'Color',
      'director_name':'UNK',
      'duration':0} 
#color 열을 Color로 대체, director_name은 범주가 너무 많으므로 모르는 것 자체로 대체
#특정 값으로 대체를 원할 경우, 이처럼 dictionary 형태로 만든 뒤, fillna를 진행하면 됨

movie = movie.fillna(col)
movie.isnull().sum()

color                          0
director_name                  0
num_critic_for_reviews        49
duration                       0
director_facebook_likes      102
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        862
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                152
movie_imdb_link                0
num_user_for_reviews          21
language                      12
country                        5
content_rating               300
budget                       484
title_year                   106
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 326
movie_facebook_likes           0
dtype: int64

In [13]:
movie = movie.fillna(movie.median())
# movie의 모든 수치형 변수는 중앙값으로 대체

In [14]:
movie = movie.dropna() #최후 결측치 제거

In [15]:
#이상치 확인은 결측치 확인이 끝난 뒤 진행
movie.describe() #현재 열이 많아서 확인하기 힘듦
movie.describe().T
movie.describe().T.iloc[:,3:] #min,max,4분위 값들만 확인 가능

,min,25%,50%,75%,max
num_critic_for_reviews,1.00,60.00,118.00,199.00,8.130000e+02
duration,0.00,94.00,104.00,118.00,3.340000e+02
director_facebook_likes,0.00,9.00,52.00,209.00,2.300000e+04
actor_3_facebook_likes,0.00,157.75,391.00,648.00,2.300000e+04
actor_1_facebook_likes,0.00,656.00,1000.00,11000.00,6.400000e+05
gross,162.00,8040422.25,25043962.00,55191053.00,7.605058e+08
num_voted_users,6.00,12072.50,39535.00,102955.25,1.689764e+06
cast_total_facebook_likes,0.00,1581.00,3299.50,14546.00,6.567300e+05
facenumber_in_poster,0.00,0.00,1.00,2.00,4.300000e+01
num_user_for_reviews,1.00,80.75,171.00,341.25,5.060000e+03


In [16]:
#duration 상 max 값이 너무 크고, min이 0인 것 또한 이상치로 확인가능
movie.duration.describe()

count    4528.000000
mean      107.944125
std        23.835007
min         0.000000
25%        94.000000
50%       104.000000
75%       118.000000
max       334.000000
Name: duration, dtype: float64

In [17]:
movie.duration.clip(lower=60, upper=180).describe() #lower나 upper 중 한 쪽만 사용 가능

count    4528.000000
mean      107.757288
std        20.632511
min        60.000000
25%        94.000000
50%       104.000000
75%       118.000000
max       180.000000
Name: duration, dtype: float64

In [18]:
c = movie.duration<=180
movie.duration.where(c, 181).describe() #조건을 넣어줘서, 조건에 맞지 않은 특정값을 변환

count    4528.000000
mean      107.483878
std        21.545534
min         0.000000
25%        94.000000
50%       104.000000
75%       118.000000
max       181.000000
Name: duration, dtype: float64

#### 파생변수

In [19]:
mem_data = pd.read_csv('./data/mem_data.csv') # 고객의 profile 데이터
mem_tr = pd.read_csv('./data/mem_transaction.csv') # 고객 구매 데이터

In [20]:
f = mem_tr.groupby('MEM_ID')['SELL_AMT'].agg('mean').reset_index()#고객별 평균 구매액
f.columns = ['MEM_ID','평균구매액']
f

,MEM_ID,평균구매액
0,957,2500.000000
1,1054,38120.000000
2,1067,7700.000000
3,1068,8255.555556
4,1078,5900.000000
...,...,...
10759,2007042901468,3300.000000
10760,2007061900497,7700.000000
10761,2007062900420,5700.000000
10762,2007090300653,32033.333333


In [21]:
mem_data = mem_data.merge(f, how='left') # 만들어낸 파생변수를 합쳐줌, how=left를 통해 좌측 데이터 프레임 기준 병합
mem_data = mem_data.fillna(0) # 결측치를 0으로 대체
mem_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10764 entries, 0 to 10763
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MEM_ID       10764 non-null  int64  
 1   M_STORE_ID   10764 non-null  int64  
 2   GENDER       10764 non-null  object 
 3   BIRTH_DT     10764 non-null  object 
 4   BIRTH_SL     10764 non-null  object 
 5   ZIP_CD       10764 non-null  object 
 6   RGST_DT      10764 non-null  object 
 7   VISIT_CNT    10764 non-null  float64
 8   SALES_AMT    10764 non-null  float64
 9   LAST_VST_DT  10764 non-null  object 
 10  USABLE_PNT   10764 non-null  float64
 11  USED_PNT     10764 non-null  float64
 12  ACC_PNT      10764 non-null  float64
 13  USABLE_INIT  10764 non-null  float64
 14  SMS          10764 non-null  object 
 15  평균구매액        10764 non-null  float64
dtypes: float64(7), int64(2), object(7)
memory usage: 1.4+ MB


In [22]:
# 요일별 구매 패턴
mem_tr.MEMP_DT = pd.to_datetime(mem_tr.MEMP_DT) #시계열 데이터 변환
mem_tr.info() # datetime으로 변한 것을 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60049 entries, 0 to 60048
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   STORE_ID  60049 non-null  int64         
 1   SELL_DT   60049 non-null  int64         
 2   MEMP_STY  60049 non-null  object        
 3   MEM_ID    60049 non-null  int64         
 4   MEMP_DT   60049 non-null  datetime64[ns]
 5   SELL_AMT  60049 non-null  int64         
 6   MEMP_TP   60049 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 3.2+ MB


In [23]:
mem_tr.MEMP_DT.dt.dayofweek #요일 추출

0        0
1        4
2        2
3        0
4        1
        ..
60044    2
60045    6
60046    4
60047    5
60048    2
Name: MEMP_DT, Length: 60049, dtype: int64

In [24]:
mem_tr['weekend'] = mem_tr.MEMP_DT.dt.dayofweek >4 #주말여부

In [27]:
f = mem_tr.groupby(['MEM_ID','weekend'])['SELL_AMT'].agg('count')
f = f.unstack().fillna(0).astype('int').reset_index()
# reset_index 보다 가장 마지막 인덱스를 열 이름으로 사용하여 True, False로 구분 
# 결측치를 0으로 바꾸고 type을 횟수이므로 int로 변환
# 마지막에 reset_index()
f.columns = ['MEM_ID','주중구매횟수','주말구매횟수']
f

,MEM_ID,주중구매횟수,주말구매횟수
0,957,3,0
1,1054,5,0
2,1067,0,1
3,1068,8,1
4,1078,3,2
...,...,...,...
10759,2007042901468,0,1
10760,2007061900497,1,0
10761,2007062900420,2,0
10762,2007090300653,3,0


In [28]:
mem_data = mem_data.merge(f, how='left')
mem_data = mem_data.fillna(0)
mem_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10764 entries, 0 to 10763
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MEM_ID       10764 non-null  int64  
 1   M_STORE_ID   10764 non-null  int64  
 2   GENDER       10764 non-null  object 
 3   BIRTH_DT     10764 non-null  object 
 4   BIRTH_SL     10764 non-null  object 
 5   ZIP_CD       10764 non-null  object 
 6   RGST_DT      10764 non-null  object 
 7   VISIT_CNT    10764 non-null  float64
 8   SALES_AMT    10764 non-null  float64
 9   LAST_VST_DT  10764 non-null  object 
 10  USABLE_PNT   10764 non-null  float64
 11  USED_PNT     10764 non-null  float64
 12  ACC_PNT      10764 non-null  float64
 13  USABLE_INIT  10764 non-null  float64
 14  SMS          10764 non-null  object 
 15  평균구매액        10764 non-null  float64
 16  주중구매횟수       10764 non-null  int32  
 17  주말구매횟수       10764 non-null  int32  
dtypes: float64(7), int32(2), int64(2), object(7)
m